In [8]:
import moldesign as mdt
from moldesign import units as u
import ipywidgets as ipy

In [2]:
from moldesign.models.lammps_model import LAMMPSPotential
import parmed as med
# mdt.configure()

In [3]:
mdt.Atom

moldesign.molecules.atoms.Atom

In [4]:
mol = mdt.from_pdb('1yu8')

In [5]:
mol = mdt.Molecule([res for res in mol.residues if res.type == "protein"])

In [6]:
mdmol = mdt.assign_forcefield(mol)

print "Done parameterizing"

Connecting to docker host at unix://var/run/docker.sock... done


Widget Javascript not detected.  It may not be installed or enabled properly.


Done parameterizing


In [9]:
rs = mdt.widgets.ResidueSelector(mdmol)
rs

In [10]:
canvas = mdmol.draw()


In [11]:
mdmol.set_energy_model(mdt.models.LAMMPSInteractive, affected_atom=mdmol.atoms[931])
mdmol.calculate()

LAMMPS output is captured by PyLammps wrapper


MolecularProperties({'positions': <Quantity([[ 10.367      29.903      18.753    ]
 [  9.6081971  30.4339133  19.1560511]
 [ 10.3127893  29.9718557  17.7468089]
 ..., 
 [  8.242      10.473      15.05     ]
 [  7.004      10.292      15.052    ]
 [  8.877      11.103      14.169    ]], 'ang')>, 'potential_energy': <Quantity(-4354.13539812, 'kcalpermol')>, 'forces': <Quantity([[-22.01450917 -39.97082631  15.02823307]
 [ -3.75360413  -4.4598381    3.53402641]
 [ 10.36030096   8.57141355   2.72603121]
 ..., 
 [ -4.10467713  11.80953405  -0.20540562]
 [  9.94705074  -5.52989577  16.52583564]
 [  2.38928098 -12.58205002  17.15230766]], 'kcalpermol / ang')>})

In [12]:
def apply_user_force(x=0.0, y=-10.0, z=0.0):
    mdmol.set_integrator(integrator=mdt.integrators.LAMMPSNvt, timestep=1.0*u.fs, 
                     temperature=300.0*u.kelvin, frame_interval=20.0*u.fs)
    mdmol.energy_model.apply_force([x, y, z])
    for step in xrange(10):
        mdmol.run(20 * u.fs)
        mdmol.energy_model.render_molecule()
        canvas.set_positions()

    mdmol.energy_model.finish_interaction()
    canvas.set_positions()
    
    mdmol.set_energy_model(mdt.models.LAMMPSInteractive, affected_atom=mdmol.atoms[931])
    mdmol.calculate()

In [13]:
canvas.set_positions()
canvas.stick()
canvas

In [14]:
ipy.interact(apply_user_force, x=ipy.FloatSlider(min=-10.0, max=10.0, step=1.0, continuous_update=False),
            y=ipy.FloatSlider(min=-10.0, max=10.0, step=1.0, continuous_update=False),
            z=ipy.FloatSlider(min=-10.0, max=10.0, step=1.0, continuous_update=False))

[0.0, -0.031131941759999997, 0.0]
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 200.0 fs to 220.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 220.0 fs to 240.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 240.0 fs to 260.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 260.0 fs to 280.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 280.0 fs to 300.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 300.0 fs to 320.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 320.0 fs to 340.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 340.0 fs to 360.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 360.0 fs to 380.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 380.0 fs to 400.0 fs
LAMMPS output is captured by PyLammps wrapper


In [48]:


# mdmol.set_integrator(integrator=mdt.integrators.LAMMPSNvt, timestep=1.0*u.fs, 
#                      temperature=300.0*u.kelvin, frame_interval=10.0*u.fs)
# mdmol.energy_model.apply_force([10.0, 0.0, 0.0])
# my_traj = mdmol.run(200 * u.fs)

# canvas.stick()
# canvas

[-0.15373798400000002, -0.038434496000000005, 0.0]
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 1200.0 fs to 1220.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 1220.0 fs to 1240.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 1240.0 fs to 1260.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 1260.0 fs to 1280.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 1280.0 fs to 1300.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 1300.0 fs to 1320.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 1320.0 fs to 1340.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 1340.0 fs to 1360.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 1360.0 fs to 1380.0 fs
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 1380.0 fs to 1400.0 fs


In [30]:
# for traj in my_traj:
#     mdmol.energy_model.render_molecule(traj)
#     canvas.set_positions(positions=traj.positions.value_in(u.angstrom))
    


In [31]:
# mdmol.energy_model.finish_interaction()
# canvas.set_positions()